In [61]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix

In [62]:
# Load the dataset
df = pd.read_csv("Alphabets_data.csv")

In [63]:
# Display basic structure
print("Dataset Summary:")
print(f"Number of samples: {df.shape[0]}")
print(f"Number of features (including target): {df.shape[1]}")
print(f"Target classes: {sorted(df['letter'].unique())}")

# Check for missing values
missing_values = df.isnull().sum().sum()
print(f"Total missing values: {missing_values}")

Dataset Summary:
Number of samples: 20000
Number of features (including target): 17
Target classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
Total missing values: 0


In [64]:
# Separate features and target
X = df.drop('letter', axis=1).values.astype(np.float32)
y = df['letter'].values

# Encode target labels (A-Z → 0–25)
label_map = {label: idx for idx, label in enumerate(sorted(np.unique(y)))}
y_encoded = np.array([label_map[label] for label in y])

# Normalize feature values to range [0, 1]
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_normalized = (X - X_min) / (X_max - X_min + 1e-7)

In [65]:
X_normalized.shape, y_encoded.shape

((20000, 16), (20000,))

In [66]:
# Randomized train-test split (80% train, 20% test)
np.random.seed(42)
indices = np.arange(len(X_normalized))
np.random.shuffle(indices)

X_shuffled = X_normalized[indices]
y_shuffled = y_encoded[indices]

split_index = int(0.8 * len(X_shuffled))
X_train = X_shuffled[:split_index]
X_test = X_shuffled[split_index:]
y_train = y_shuffled[:split_index]
y_test = y_shuffled[split_index:]

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Wrap into TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [67]:
class AlphabetClassifier(nn.Module):
    def __init__(self, input_size=16, hidden1=64, hidden2=32, output_size=26, activation=nn.ReLU):
        super(AlphabetClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.act1 = activation()
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.act2 = activation()
        self.fc3 = nn.Linear(hidden2, output_size)

    def forward(self, x):
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))
        x = self.fc3(x)
        return x

In [68]:
# Instantiate the model
model = AlphabetClassifier()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/10, Loss: 2.9609
Epoch 2/10, Loss: 1.9727
Epoch 3/10, Loss: 1.5995
Epoch 4/10, Loss: 1.4510
Epoch 5/10, Loss: 1.3627
Epoch 6/10, Loss: 1.2954
Epoch 7/10, Loss: 1.2432
Epoch 8/10, Loss: 1.1913
Epoch 9/10, Loss: 1.1459
Epoch 10/10, Loss: 1.1036


In [69]:
# Evaluation on test data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 67.85%


In [70]:
# Collect predictions
y_pred = []
y_true = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())
        y_true.extend(labels.numpy())

# Print metrics
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       150
           1       0.60      0.77      0.68       169
           2       0.92      0.72      0.81       160
           3       0.72      0.56      0.63       165
           4       0.66      0.54      0.60       161
           5       0.74      0.56      0.64       167
           6       0.55      0.50      0.52       144
           7       0.57      0.15      0.24       156
           8       0.78      0.83      0.81       145
           9       0.78      0.71      0.75       132
          10       0.56      0.76      0.64       141
          11       0.95      0.74      0.83       172
          12       0.86      0.88      0.87       160
          13       0.64      0.82      0.72       165
          14       0.61      0.69      0.64       150
          15       0.76      0.69      0.72       172
          16       0.57      0.71      0.63       160
          17       0.60    

In [71]:
# Define activation functions to try
activations = [nn.ReLU, nn.Tanh, nn.Sigmoid]

# Define configurations to try (each activation × 4 configs)
configs = [
    {"hidden1": 64, "hidden2": 32, "lr": 0.001, "epochs": 10, "activation": act}
    for act in activations
] + [
    {"hidden1": 128, "hidden2": 64, "lr": 0.001, "epochs": 10, "activation": act}
    for act in activations
] + [
    {"hidden1": 128, "hidden2": 64, "lr": 0.0005, "epochs": 15, "activation": act}
    for act in activations
] + [
    {"hidden1": 64, "hidden2": 64, "lr": 0.005, "epochs": 10, "activation": act}
    for act in activations
]

# Store results
results = []

for i, config in enumerate(configs):
    print(f"\n🔧 Config {i+1}: {config}, Activation: {config['activation'].__name__}")

    # Define model with activation
    model = AlphabetClassifier(
        hidden1=config["hidden1"],
        hidden2=config["hidden2"],
        activation=config["activation"]
    )
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    # Train
    for epoch in range(config["epochs"]):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"  Epoch {epoch+1}/{config['epochs']}, Loss: {avg_loss:.4f}")

    # Evaluate
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    results.append((config, accuracy))


🔧 Config 1: {'hidden1': 64, 'hidden2': 32, 'lr': 0.001, 'epochs': 10, 'activation': <class 'torch.nn.modules.activation.ReLU'>}, Activation: ReLU
  Epoch 1/10, Loss: 3.0114
  Epoch 2/10, Loss: 2.0457
  Epoch 3/10, Loss: 1.6696
  Epoch 4/10, Loss: 1.5156
  Epoch 5/10, Loss: 1.4169
  Epoch 6/10, Loss: 1.3455
  Epoch 7/10, Loss: 1.2888
  Epoch 8/10, Loss: 1.2372
  Epoch 9/10, Loss: 1.1909
  Epoch 10/10, Loss: 1.1469
Test Accuracy: 67.35%

🔧 Config 2: {'hidden1': 64, 'hidden2': 32, 'lr': 0.001, 'epochs': 10, 'activation': <class 'torch.nn.modules.activation.Tanh'>}, Activation: Tanh
  Epoch 1/10, Loss: 2.8112
  Epoch 2/10, Loss: 1.8209
  Epoch 3/10, Loss: 1.4869
  Epoch 4/10, Loss: 1.3199
  Epoch 5/10, Loss: 1.2012
  Epoch 6/10, Loss: 1.1082
  Epoch 7/10, Loss: 1.0322
  Epoch 8/10, Loss: 0.9695
  Epoch 9/10, Loss: 0.9169
  Epoch 10/10, Loss: 0.8751
Test Accuracy: 74.05%

🔧 Config 3: {'hidden1': 64, 'hidden2': 32, 'lr': 0.001, 'epochs': 10, 'activation': <class 'torch.nn.modules.activation

In [72]:
best_config = max(results, key=lambda x: x[1])
print("\nBest configuration and accuracy:")
print(best_config)


Best configuration and accuracy:
({'hidden1': 64, 'hidden2': 64, 'lr': 0.005, 'epochs': 10, 'activation': <class 'torch.nn.modules.activation.Tanh'>}, 89.6)


## Discussion and Conclusion

In this assignment, we built an Artificial Neural Network (ANN) to classify alphabet data and studied how tuning different hyperparameters—such as hidden layer sizes, learning rate, number of epochs, and activation functions—affects its accuracy. The base model started with 66.40% accuracy. After tuning, the best-performing configuration used two hidden layers of 64 neurons each, a learning rate of 0.005, 10 epochs, and the Tanh activation function, achieving a final accuracy of 86.27%. These results demonstrate that careful tuning of model architecture and activation functions significantly improves learning and classification performance.